In [ ]:
path = []


In [2]:
path = [[0, 10], [1, 9], [2, 9], [3, 10], [4, 9], [5, 10], [6, 9], [7, 9], [8, 10], [9, 10], [10, 11], [11, 10], [12, 10], [13, 11],[14, 11], [15, 11], [16, 10], [17, 10], [18, 9], [19, 10]]

In [19]:
import cv2
import numpy as np
blank = cv2.imread("blank.png",0)
blank = cv2.resize(blank,(20,20))
# path = np.array(path)
# epsilon = 0.1*cv2.arcLength(path, True)
# path = cv2.approxPolyDP(path, epsilon, False)
# print(path)
pnts = []
for i in range(1,16):
    pts = [np.mean(path[0:15*(i)][0]),np.mean(path[0:15*(i)][1])]
    pnts.append(pts)
#     cv2.line(blank,(path[i][1],path[i][0]),(path[i+1][1],path[i+1][0]),0)
#     pnts = np.mean(path[i:i+15][0])
print(pnts)
blank = cv2.resize(blank,(680,480))
cv2.imshow("blank",blank)
q = cv2.waitKey(1)
if q == 27:
    cv2.destroyAllWindows()

[[5.0, 5.0], [7.5, 7.5], [10.5, 10.5]]


In [32]:
import math
slp = []
for pix in pnts:
    slp.append(math.atan2(340-pix[1],pix[0]-240) - 1.57)

In [33]:
import math
print(slp)

[1.1292184306130484, 0.0007963267948964958]


In [ ]:
#output:: 6 category  
#input:: 15 slope 
#mode = xgboost


In [74]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

EPISODES = 500


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=pow(10,20))
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
    
    def _huber_loss(self, target, prediction):
        error = prediction - target
        return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss, optimizer=Adam(lr=self.learning_rate))
        return model
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0]) 
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)
    
    def save(self, name):
        self.model.save_weights(name)
    
    def decl_rew(self,state):
        rew = np.sum(([1.75]*10 - abs(state))*[10,10,10,9,9,9,8,8,7,6,5,4,3,2,1])
        d = False
        if rew > 130:
            d = True
        return [nxtstate, rew, True]
        


def steer():
    state_size = 15
    action_size = 6
    agent = DQNAgent(state_size, action_size)
    
    agent.load("./save/steerwts.h5")
    
    batch_size = 50
    
    while True:
        done = False
        while True:
            state = inp_state 
            state = np.reshape(state, [1, state_size])
            action = agent.act(state)
            print("action:",action)
            execute_action(action)
            next_state, reward, done, _ = decl_rew(state)##input and reward
            if(next_state == []):
                break;
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if abs(np.mean(state)) <= 0.175:
                done = True
            if done or upd:
                print("updating weights under 10 deg")
                agent.update_target_model()
                print("updated")
                done = False
            if len(agent.memory) > batch_size:
                print("replaying at 50 and saving weights")
                agent.save("./save/steerwts.h5")
                print("weights saved")
                agent.replay(batch_size)
                print("replayed")

In [49]:
state = env.reset()
print(state_size)
state = np.reshape(state, [-1, state_size])
print(state)

4
[[-0.02340593 -0.04727202 -0.00067567 -0.02899503]]


In [50]:
# agent.save("./save/cartpole-ddqn.h5")
print(state_size)

4


In [12]:
import numpy as np
from PIL import ImageGrab
import cv2
import time

def screen_record(): 
    last_time = time.time()
    while True:
        # 800x600 windowed mode
        printscreen_pil =  ImageGrab.grab(bbox=(0,40,800,640))
        printscreen_numpy =   np.array(printscreen_pil.getdata(),dtype='uint8')\
        .reshape((printscreen_pil.size[1],printscreen_pil.size[0],3))
        print('loop took {} seconds'.format(time.time()-last_time))
        last_time = time.time()

    ##    cv2.imshow('window',cv2.cvtColor(printscreen_numpy, cv2.COLOR_BGR2RGB))
    ##    if cv2.waitKey(25) & 0xFF == ord('q'):
    ##        cv2.destroyAllWindows()
    ##        break

In [52]:
import math
print(pow(10,100))

10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [73]:
a=[]
a is None

False

In [75]:
[1.75]*10

[1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75]